# 10강: ETF 기본 & 포트폴리오 지표 계산

이 강의에서는 9강에서 저장한 로우 데이터를 읽어서
**5강(기본 지표)**, **6강(포트폴리오 지표)** 내용을 **숫자로 구현**합니다.

## 이 강의의 목표

1. 9강에서 수집한 CSV 파일 로딩
2. 추가 메타데이터 API 호출 (종목별 섹터, 시가총액, PER/PBR 등)
3. 기본 지표 계산 (AUM, 규모, 설정일, 유동성)
4. 포트폴리오 지표 계산 (집중도, 섹터/시가총액 분포, 가중평균 밸류에이션)

## 입력 데이터 (9강에서 저장)

| 파일 | 용도 |
|------|------|
| `data/raw/etf_info.csv` | ETF 기본 정보 |
| `data/raw/holdings_{ticker}.csv` | ETF 구성종목 |
| `data/raw/price_{ticker}.csv` | 평균 거래량 계산용 |

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import date, datetime
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple

from kis_utils import (
    DATA_DIR,
    to_float,
    ensure_kis_token,
    call_kis_get,
    get_etf_name_from_kis,
)

## Step 0. 디렉터리 설정 및 데이터 로딩

In [2]:
# 디렉터리 설정
RAW_DATA_DIR = DATA_DIR / "raw"
COMPONENT_DIR = DATA_DIR / "etf_components"
SUMMARY_DIR = DATA_DIR / "etf_portfolio_summary"

COMPONENT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_DIR.mkdir(parents=True, exist_ok=True)

print(f"RAW_DATA_DIR : {RAW_DATA_DIR}")
print(f"COMPONENT_DIR: {COMPONENT_DIR}")
print(f"SUMMARY_DIR  : {SUMMARY_DIR}")

RAW_DATA_DIR : /Users/ulift/workspace/py-etf-mango/script2/claude/data/raw
COMPONENT_DIR: /Users/ulift/workspace/py-etf-mango/script2/claude/data/etf_components
SUMMARY_DIR  : /Users/ulift/workspace/py-etf-mango/script2/claude/data/etf_portfolio_summary


In [3]:
def load_etf_info() -> pd.DataFrame:
    """9강에서 저장한 ETF 기본 정보 로딩."""
    path = RAW_DATA_DIR / "etf_info.csv"
    if not path.exists():
        raise FileNotFoundError(f"ETF 정보 파일이 없습니다: {path}")
    return pd.read_csv(path)


def load_holdings(etf_code: str) -> pd.DataFrame:
    """9강에서 저장한 ETF 구성종목 로딩."""
    etf_code = str(etf_code).strip().zfill(6)
    path = RAW_DATA_DIR / f"holdings_{etf_code}.csv"
    if not path.exists():
        raise FileNotFoundError(f"구성종목 파일이 없습니다: {path}")
    return pd.read_csv(path)


def load_price(etf_code: str) -> pd.DataFrame:
    """9강에서 저장한 ETF 가격 시계열 로딩."""
    etf_code = str(etf_code).strip().zfill(6)
    path = RAW_DATA_DIR / f"price_{etf_code}.csv"
    if not path.exists():
        raise FileNotFoundError(f"가격 시계열 파일이 없습니다: {path}")
    df = pd.read_csv(path)
    df["date"] = pd.to_datetime(df["date"])
    return df

---

## Step 1. 추가 메타데이터 수집

포트폴리오 지표 계산을 위해 종목별로 추가 정보를 조회합니다.

### 1-1. 주식기본조회 API (`CTPF1002R`)

**수집 필드:** 섹터(표준산업분류), 업종 대/중분류, 상장주수

In [4]:
def fetch_stock_basic_batch(
    codes: List[str],
    *,
    sleep_sec: float = 0.05
) -> pd.DataFrame:
    """
    여러 종목코드에 대해 '주식기본조회' API 호출.
    섹터/상장주수 등 기본 정보를 수집.
    """
    path = "/uapi/domestic-stock/v1/quotations/search-stock-info"
    tr_id = "CTPF1002R"
    
    records = []
    
    for idx, code in enumerate(codes, start=1):
        code = str(code).strip().zfill(6)
        print(f"[{idx}/{len(codes)}] 주식기본조회: {code}")
        
        params = {
            "PRDT_TYPE_CD": "300",
            "PDNO": code,
        }
        
        try:
            data = call_kis_get(path, tr_id, params)
        except Exception as e:
            print(f"  ⚠️ 실패: {e}")
            continue
        
        output = data.get("output") or {}
        if not output:
            continue
        
        row = {
            "stck_shrn_iscd": code,
            "prdt_name": output.get("prdt_name", "").strip(),
            "std_idst_clsf_cd_name": output.get("std_idst_clsf_cd_name", "").strip(),
            "idx_bztp_lcls_cd_name": output.get("idx_bztp_lcls_cd_name", "").strip(),
            "idx_bztp_mcls_cd_name": output.get("idx_bztp_mcls_cd_name", "").strip(),
            "lstg_stqt": output.get("lstg_stqt"),
        }
        records.append(row)
        time.sleep(sleep_sec)
    
    if not records:
        return pd.DataFrame()
    
    df = pd.DataFrame(records)
    df["lstg_stqt_float"] = df["lstg_stqt"].apply(to_float)
    
    return df

### 1-2. 주식현재가 시세 API (`FHKST01010100`)

**수집 필드:** PER, PBR, EPS, BPS, 시가총액, 52주 최고/최저

In [5]:
def fetch_stock_current_price(code: str) -> Dict[str, Any]:
    """
    주식현재가 시세 API 호출.
    PER, PBR, 시가총액 등 밸류에이션 정보 반환.
    """
    path = "/uapi/domestic-stock/v1/quotations/inquire-price"
    tr_id = "FHKST01010100"
    
    code = str(code).strip().zfill(6)
    
    params = {
        "FID_COND_MRKT_DIV_CODE": "J",
        "FID_INPUT_ISCD": code,
    }
    
    try:
        data = call_kis_get(path, tr_id, params)
    except Exception as e:
        print(f"  ⚠️ 주식현재가 실패 ({code}): {e}")
        return {}
    
    output = data.get("output") or {}
    
    return {
        "code": code,
        "stck_prpr": to_float(output.get("stck_prpr")),
        "hts_avls": to_float(output.get("hts_avls")),  # 시가총액
        "per": to_float(output.get("per")),
        "pbr": to_float(output.get("pbr")),
        "eps": to_float(output.get("eps")),
        "bps": to_float(output.get("bps")),
        "w52_hgpr": to_float(output.get("w52_hgpr")),
        "w52_lwpr": to_float(output.get("w52_lwpr")),
    }


def fetch_stock_current_price_batch(
    codes: List[str],
    *,
    sleep_sec: float = 0.05
) -> pd.DataFrame:
    """여러 종목의 현재가/밸류에이션 정보 배치 조회."""
    records = []
    
    for idx, code in enumerate(codes, start=1):
        print(f"[{idx}/{len(codes)}] 주식현재가: {code}")
        result = fetch_stock_current_price(code)
        if result:
            records.append(result)
        time.sleep(sleep_sec)
    
    if not records:
        return pd.DataFrame()
    
    return pd.DataFrame(records)

---

## Step 2. 집중도 지표 (6강 연계)

포트폴리오의 집중도를 측정하는 지표들입니다.

| 지표 | 설명 | 해석 |
|------|------|------|
| Top5/Top10 | 상위 N개 종목 비중 합 | 높을수록 집중 |
| HHI | 허핀달 지수 (Σw²) | 0~1, 높을수록 집중 |
| Effective N | 유효 종목 수 (1/HHI) | 낮을수록 집중 |

In [6]:
def calc_top_n_concentration(df: pd.DataFrame, n: int = 10) -> float:
    """상위 N개 종목 비중 합계."""
    if "weight" not in df.columns:
        return float("nan")
    weights = df["weight"].dropna().sort_values(ascending=False)
    return float(weights.head(n).sum())


def calc_herfindahl_index(df: pd.DataFrame) -> float:
    """
    허핀달 지수 (HHI) = Σ(weight)²
    - 0에 가까울수록 분산
    - 1에 가까울수록 집중
    """
    if "weight" not in df.columns:
        return float("nan")
    weights = df["weight"].dropna()
    if weights.empty:
        return float("nan")
    return float((weights ** 2).sum())


def calc_effective_n(df: pd.DataFrame) -> float:
    """
    유효 종목 수 = 1 / HHI
    동일 비중이면 N, 집중되면 1에 가까움
    """
    hhi = calc_herfindahl_index(df)
    if hhi <= 0 or np.isnan(hhi):
        return float("nan")
    return float(1 / hhi)

---

## Step 3. 시가총액 분류 & 분포 (6강 연계)

In [7]:
def classify_size_bucket(market_caps: pd.Series) -> pd.Series:
    """
    시가총액 기준 Large/Mid/Small 분류.
    - 상위 30%: Large
    - 하위 30%: Small
    - 나머지: Mid
    
    *주의*: ETF 내부 상대 크기 기준입니다.
    """
    s = market_caps.fillna(0).copy()
    
    if len(s) == 0 or s.max() <= 0:
        return pd.Series(["Unknown"] * len(s), index=s.index)
    
    q_low = s.quantile(0.3)
    q_high = s.quantile(0.7)
    
    def _bucket(x):
        if x <= 0:
            return "Unknown"
        if x >= q_high:
            return "Large"
        if x <= q_low:
            return "Small"
        return "Mid"
    
    return s.apply(_bucket)


def calc_size_distribution(df: pd.DataFrame) -> pd.DataFrame:
    """시가총액 버킷별 비중 분포."""
    if "weight" not in df.columns or "size_bucket" not in df.columns:
        return pd.DataFrame()
    
    result = (
        df.dropna(subset=["weight"])
        .groupby("size_bucket", dropna=False)["weight"]
        .sum()
        .reset_index()
    )
    result.rename(columns={"weight": "weight_sum"}, inplace=True)
    return result

---

## Step 4. 섹터 분포 (6강 연계)

In [8]:
def calc_sector_distribution(df: pd.DataFrame) -> pd.DataFrame:
    """섹터별 비중 분포."""
    if "weight" not in df.columns:
        return pd.DataFrame()
    
    # 섹터 컬럼 확인 (std_idst_clsf_cd_name 우선)
    sector_col = None
    for col in ["std_idst_clsf_cd_name", "idx_bztp_mcls_cd_name", "sector"]:
        if col in df.columns:
            sector_col = col
            break
    
    if not sector_col:
        return pd.DataFrame()
    
    result = (
        df.dropna(subset=["weight"])
        .groupby(sector_col, dropna=False)["weight"]
        .sum()
        .reset_index()
    )
    result.rename(columns={sector_col: "sector", "weight": "weight_sum"}, inplace=True)
    return result.sort_values("weight_sum", ascending=False)

---

## Step 5. 가중평균 밸류에이션 (6강 연계)

In [9]:
def calc_portfolio_weighted_metrics(df: pd.DataFrame) -> Dict[str, float]:
    """
    포트폴리오 가중평균 PER, PBR, 시가총액 계산.
    """
    if "weight" not in df.columns:
        return {}
    
    df = df.dropna(subset=["weight"]).copy()
    result = {}
    
    # 가중평균 PER
    if "per" in df.columns:
        df["per_float"] = pd.to_numeric(df["per"], errors="coerce")
        valid = df[(df["per_float"] > 0) & (df["per_float"] < 200)].dropna(subset=["per_float"])
        if not valid.empty and valid["weight"].sum() > 0:
            weighted_per = (valid["per_float"] * valid["weight"]).sum() / valid["weight"].sum()
            result["weighted_avg_per"] = float(weighted_per)
    
    # 가중평균 PBR
    if "pbr" in df.columns:
        df["pbr_float"] = pd.to_numeric(df["pbr"], errors="coerce")
        valid = df[(df["pbr_float"] > 0) & (df["pbr_float"] < 50)].dropna(subset=["pbr_float"])
        if not valid.empty and valid["weight"].sum() > 0:
            weighted_pbr = (valid["pbr_float"] * valid["weight"]).sum() / valid["weight"].sum()
            result["weighted_avg_pbr"] = float(weighted_pbr)
    
    # 가중평균 시가총액
    if "market_cap" in df.columns:
        valid = df.dropna(subset=["market_cap"])
        if not valid.empty and valid["weight"].sum() > 0:
            weighted_mcap = (valid["market_cap"] * valid["weight"]).sum() / valid["weight"].sum()
            result["weighted_avg_market_cap"] = float(weighted_mcap)
    
    return result

---

## Step 6. 회전율 계산 (6강 연계)

포트폴리오 회전율은 이전 스냅샷과 현재 비중을 비교하여 계산합니다.

$$\text{Turnover} = \frac{\sum |w_t - w_{t-1}|}{2}$$

In [10]:
def calc_turnover(current_df: pd.DataFrame, prev_df: pd.DataFrame) -> float:
    """
    현재와 이전 스냅샷의 비중을 비교하여 회전율 계산.
    """
    if "weight" not in current_df.columns or "weight" not in prev_df.columns:
        return 0.0
    
    # 종목코드 기준 병합
    curr_w = current_df.set_index("stck_shrn_iscd")["weight"]
    prev_w = prev_df.set_index("stck_shrn_iscd")["weight"]
    
    combined = pd.DataFrame({"curr": curr_w, "prev": prev_w}).fillna(0)
    
    # 회전율 = |변화량 합| / 2
    diff_sum = (combined["curr"] - combined["prev"]).abs().sum()
    turnover = diff_sum / 2.0
    
    return float(turnover)


def find_previous_holdings(etf_code: str, current_date: str) -> Optional[pd.DataFrame]:
    """이전 날짜의 구성종목 파일을 찾아 로딩."""
    pattern = f"{etf_code}_*_components.csv"
    candidates = sorted(COMPONENT_DIR.glob(pattern))
    
    for path in reversed(candidates):
        if current_date not in path.name:
            try:
                df = pd.read_csv(path)
                df["stck_shrn_iscd"] = df["stck_shrn_iscd"].astype(str).str.zfill(6)
                return df
            except Exception:
                continue
    
    return None

---

## Step 7. ETF 유형 플래그 파싱 (5강/8강 연계)

ETF 구분명에서 레버리지/인버스/배당형 등을 판별합니다.

In [11]:
def parse_etf_type_flags(etf_div_name: str, etf_name: str = "") -> Dict[str, Any]:
    """
    ETF 구분명과 이름에서 특수 ETF 플래그를 파싱.
    
    반환 dict:
    - is_leveraged: 레버리지 여부
    - leverage_factor: 배율 (1, 2, 3)
    - is_inverse: 인버스 여부
    - is_covered_call: 커버드콜 여부
    - is_dividend_focused: 배당형 여부
    - asset_class: EQUITY / BOND / MIXED / COMMODITY / REITS / OTHER
    """
    text = f"{etf_div_name} {etf_name}".upper()
    
    # 레버리지
    is_leveraged = "레버리지" in text or "LEVERAGE" in text or "2X" in text or "3X" in text
    leverage_factor = 1
    if "3X" in text or "3배" in text:
        leverage_factor = 3
    elif "2X" in text or "2배" in text or "레버리지" in text:
        leverage_factor = 2
    
    # 인버스
    is_inverse = "인버스" in text or "INVERSE" in text or "숏" in text or "SHORT" in text
    
    # 커버드콜
    is_covered_call = "커버드콜" in text or "COVERED" in text or "프리미엄" in text
    
    # 배당형
    dividend_keywords = ["배당", "고배당", "DIVIDEND", "배당성장", "월배당"]
    is_dividend_focused = any(kw in text for kw in dividend_keywords)
    
    # 자산 클래스 추론
    asset_class = "EQUITY"
    if any(kw in text for kw in ["채권", "BOND", "국채", "회사채", "단기채"]):
        asset_class = "BOND"
    elif any(kw in text for kw in ["혼합", "TDF", "자산배분", "멀티에셋"]):
        asset_class = "MIXED"
    elif any(kw in text for kw in ["금", "원유", "GOLD", "OIL", "커머디티", "원자재"]):
        asset_class = "COMMODITY"
    elif any(kw in text for kw in ["리츠", "REIT", "부동산"]):
        asset_class = "REITS"
    
    return {
        "is_leveraged": is_leveraged,
        "leverage_factor": leverage_factor,
        "is_inverse": is_inverse,
        "is_covered_call": is_covered_call,
        "is_dividend_focused": is_dividend_focused,
        "asset_class": asset_class,
    }


def calc_aum_tier(aum: float) -> Dict[str, Any]:
    """
    AUM 기준 규모 분류.
    - LARGE: 1조 이상
    - MEDIUM: 500억 ~ 1조
    - SMALL: 100억 ~ 500억
    - MICRO: 100억 미만
    """
    if aum is None or aum <= 0:
        return {"is_small_aum_risk": True, "aum_tier": "UNKNOWN"}
    
    threshold = 500_0000_0000  # 500억원
    is_small_aum_risk = aum < threshold
    
    if aum >= 1_0000_0000_0000:  # 1조 이상
        aum_tier = "LARGE"
    elif aum >= 500_0000_0000:   # 500억 이상
        aum_tier = "MEDIUM"
    elif aum >= 100_0000_0000:   # 100억 이상
        aum_tier = "SMALL"
    else:
        aum_tier = "MICRO"
    
    return {
        "is_small_aum_risk": is_small_aum_risk,
        "aum_tier": aum_tier,
    }

---

## Step 8. 전체 포트폴리오 스냅샷 빌드

위 함수들을 조합하여 하나의 ETF에 대한 전체 분석을 수행합니다.

In [12]:
def build_etf_portfolio_snapshot(etf_code: str) -> Dict[str, Any]:
    """
    하나의 ETF에 대해 전체 포트폴리오 스냅샷 생성.
    
    반환:
    - components: 구성종목 상세 (비중, 섹터, 시총, PER/PBR 포함)
    - sector_summary: 섹터별 비중 요약
    - size_summary: 시가총액 버킷별 비중 요약
    - meta: ETF 메타 정보 (집중도 지표, 가중평균 밸류에이션 등)
    """
    etf_code = str(etf_code).strip().zfill(6)
    as_of_date = date.today().isoformat()
    
    print(f"\n{'='*50}")
    print(f"ETF 포트폴리오 스냅샷 빌드: {etf_code}")
    print(f"{'='*50}")
    
    # 1) 기존 데이터 로딩
    print("\n[1] 데이터 로딩...")
    df_holdings = load_holdings(etf_code)
    df_etf_info = load_etf_info()
    etf_info_row = df_etf_info[df_etf_info["ticker"] == etf_code]
    
    # 2) 종목 리스트 추출
    codes = sorted(df_holdings["stck_shrn_iscd"].dropna().unique().tolist())
    print(f"   구성종목 수: {len(codes)}")
    
    # 3) 추가 메타 데이터 조회
    print("\n[2] 추가 메타 데이터 조회...")
    df_basic = fetch_stock_basic_batch(codes)
    df_price_info = fetch_stock_current_price_batch(codes)
    
    # 4) 데이터 병합
    print("\n[3] 데이터 병합...")
    df = df_holdings.copy()
    
    # 기본정보 병합
    if not df_basic.empty:
        merge_cols = [c for c in ["stck_shrn_iscd", "std_idst_clsf_cd_name", 
                                   "idx_bztp_lcls_cd_name", "idx_bztp_mcls_cd_name",
                                   "lstg_stqt_float"] if c in df_basic.columns]
        df = df.merge(df_basic[merge_cols], on="stck_shrn_iscd", how="left")
    
    # 밸류에이션 병합
    if not df_price_info.empty:
        df_price_info = df_price_info.rename(columns={"code": "stck_shrn_iscd"})
        merge_cols = [c for c in ["stck_shrn_iscd", "per", "pbr", "eps", "bps", 
                                   "hts_avls", "w52_hgpr", "w52_lwpr"] if c in df_price_info.columns]
        df = df.merge(df_price_info[merge_cols], on="stck_shrn_iscd", how="left")
    
    # 5) 시가총액 계산
    if "hts_avls" in df.columns:
        df["market_cap"] = pd.to_numeric(df["hts_avls"], errors="coerce")
    else:
        df["market_cap"] = None
    
    # 6) Size bucket 분류
    if "market_cap" in df.columns:
        df["size_bucket"] = classify_size_bucket(df["market_cap"])
    
    # 7) 분포 계산
    print("\n[4] 지표 계산...")
    sector_summary = calc_sector_distribution(df)
    sector_summary["as_of_date"] = as_of_date
    sector_summary["etf_code"] = etf_code
    
    size_summary = calc_size_distribution(df)
    size_summary["as_of_date"] = as_of_date
    size_summary["etf_code"] = etf_code
    
    # 8) 집중도 지표
    top5_conc = calc_top_n_concentration(df, n=5)
    top10_conc = calc_top_n_concentration(df, n=10)
    hhi = calc_herfindahl_index(df)
    effective_n = calc_effective_n(df)
    
    # 9) 가중평균 밸류에이션
    weighted_metrics = calc_portfolio_weighted_metrics(df)
    
    # 10) ETF 유형 플래그
    etf_name = get_etf_name_from_kis(etf_code) or etf_code
    etf_div_name = ""
    if not etf_info_row.empty:
        etf_div_name = etf_info_row.iloc[0].get("etf_div_name", "") or ""
    type_flags = parse_etf_type_flags(etf_div_name, etf_name)
    
    # 11) AUM 티어
    aum_val = None
    if not etf_info_row.empty:
        aum_val = to_float(etf_info_row.iloc[0].get("aum"))
    aum_flags = calc_aum_tier(aum_val)
    
    # 12) 회전율 (이전 스냅샷 대비)
    prev_df = find_previous_holdings(etf_code, as_of_date)
    turnover = 0.0
    if prev_df is not None:
        turnover = calc_turnover(df, prev_df)
        print(f"   회전율: {turnover:.4f}")
    else:
        print("   회전율: 이전 데이터 없음 (0.0)")
    
    # 13) 메타 정보 조합
    meta = pd.DataFrame([{
        "as_of_date": as_of_date,
        "etf_code": etf_code,
        "etf_name": etf_name,
        
        # 기본 정보
        "mbcr_name": etf_info_row.iloc[0].get("mbcr_name") if not etf_info_row.empty else None,
        "listing_date": etf_info_row.iloc[0].get("listing_date") if not etf_info_row.empty else None,
        "etf_div_name": etf_div_name,
        
        # 유형 플래그
        **type_flags,
        **aum_flags,
        
        # 구성종목 수
        "n_components": len(df),
        
        # 집중도 지표
        "top5_concentration": top5_conc if not np.isnan(top5_conc) else None,
        "top10_concentration": top10_conc if not np.isnan(top10_conc) else None,
        "herfindahl_index": hhi if not np.isnan(hhi) else None,
        "effective_n": effective_n if not np.isnan(effective_n) else None,
        
        # 가중평균 밸류에이션
        **weighted_metrics,
        
        # 회전율
        "turnover": turnover,
    }])
    
    print("\n[5] 완료!")
    
    return {
        "components": df,
        "sector_summary": sector_summary,
        "size_summary": size_summary,
        "meta": meta,
    }

---

## Step 9. 실행 및 저장

In [13]:
def run_etf_portfolio_job(etf_code: str) -> Dict[str, pd.DataFrame]:
    """
    ETF 포트폴리오 분석 실행 및 파일 저장.
    """
    result = build_etf_portfolio_snapshot(etf_code)
    
    df_components = result["components"]
    df_sector = result["sector_summary"]
    df_size = result["size_summary"]
    df_meta = result["meta"]
    
    as_of_date = df_meta["as_of_date"].iloc[0]
    
    # 파일 저장
    comp_path = COMPONENT_DIR / f"{etf_code}_{as_of_date}_components.csv"
    sector_path = SUMMARY_DIR / f"{etf_code}_{as_of_date}_sector.csv"
    size_path = SUMMARY_DIR / f"{etf_code}_{as_of_date}_size.csv"
    meta_path = SUMMARY_DIR / f"{etf_code}_{as_of_date}_meta.csv"
    
    df_components.to_csv(comp_path, index=False, encoding="utf-8-sig")
    df_sector.to_csv(sector_path, index=False, encoding="utf-8-sig")
    df_size.to_csv(size_path, index=False, encoding="utf-8-sig")
    df_meta.to_csv(meta_path, index=False, encoding="utf-8-sig")
    
    print(f"\n[SAVE] components → {comp_path}")
    print(f"[SAVE] sector     → {sector_path}")
    print(f"[SAVE] size       → {size_path}")
    print(f"[SAVE] meta       → {meta_path}")
    
    return result

---

## 예시 실행: KODEX 200

In [ ]:
# 토큰 확인
ensure_kis_token()

# KODEX 200 분석 실행
result = run_etf_portfolio_job("069500")

print("\n" + "="*60)
print("결과 미리보기")
print("="*60)


ETF 포트폴리오 스냅샷 빌드: 069500

[1] 데이터 로딩...
   구성종목 수: 100

[2] 추가 메타 데이터 조회...
[1/100] 주식기본조회: 000100
[2/100] 주식기본조회: 000150
[3/100] 주식기본조회: 000270
[4/100] 주식기본조회: 000660
[5/100] 주식기본조회: 000720
[6/100] 주식기본조회: 000810
[7/100] 주식기본조회: 000880
[8/100] 주식기본조회: 001040
[9/100] 주식기본조회: 001440
[10/100] 주식기본조회: 003230
[11/100] 주식기본조회: 003490
[12/100] 주식기본조회: 003550
[13/100] 주식기본조회: 003670
[14/100] 주식기본조회: 004020
[15/100] 주식기본조회: 005380
[16/100] 주식기본조회: 005490
[17/100] 주식기본조회: 005830
[18/100] 주식기본조회: 005930
[19/100] 주식기본조회: 005940
[20/100] 주식기본조회: 006260
[21/100] 주식기본조회: 006400
[22/100] 주식기본조회: 006800
[23/100] 주식기본조회: 009150
[24/100] 주식기본조회: 009420
[25/100] 주식기본조회: 009540
[26/100] 주식기본조회: 009830
[27/100] 주식기본조회: 010120
[28/100] 주식기본조회: 010130
[29/100] 주식기본조회: 010140
[30/100] 주식기본조회: 010950
[31/100] 주식기본조회: 011070
[32/100] 주식기본조회: 011200
[33/100] 주식기본조회: 011790
[34/100] 주식기본조회: 012330
[35/100] 주식기본조회: 012450
[36/100] 주식기본조회: 0126Z0
[37/100] 주식기본조회: 014680
[38/100] 주식기본조회: 015760
[39/100] 주식기본조회: 016

In [ ]:
# 구성종목 (상위 5개)
print("\n[구성종목 - 상위 5개]")
display(result["components"].nlargest(5, "weight")[
    ["stck_shrn_iscd", "hts_kor_isnm", "weight", "per", "pbr", "size_bucket"]
])

# 섹터 분포
print("\n[섹터 분포 - 상위 5개]")
display(result["sector_summary"].head())

# 시가총액 분포
print("\n[시가총액 분포]")
display(result["size_summary"])

# 메타 정보
print("\n[메타 정보]")
display(result["meta"].T)

---

## 정리: 10강에서 계산한 지표들

### 5강 연계 - 기본 정보 지표
- AUM 티어 (LARGE/MEDIUM/SMALL/MICRO)
- 운용사명, 상장일
- ETF 유형 플래그 (레버리지/인버스/배당형 등)

### 6강 연계 - 포트폴리오 지표
- **집중도**: Top5, Top10 비중 / HHI / Effective N
- **섹터 분포**: 표준산업분류 기준 비중
- **시가총액 분포**: Large/Mid/Small 비중
- **밸류에이션**: 가중평균 PER, PBR
- **회전율**: 이전 스냅샷 대비 비중 변화

### 📌 11강에서 이어질 내용
- 수익률 계산 (기간별, YTD, 설정일 이후)
- 리스크 지표 (변동성, MDD, Sharpe, Sortino)
- 상대 지표 (Beta, Alpha, 추적오차)